<a href="https://colab.research.google.com/github/yagneshmb/Fake_News_Detection_Deep_Learnng/blob/main/hindawi_mp_6_apr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Install dependencies

In [ ]:
# !pip install https://github.com/CellProfiling/HPA-Cell-Segmentation/archive/master.zip

Read data from excel file

In [ ]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/mp_hindawi/train.xlsx', 'true_false_data') # can also index sheet by name or fetch all sheets
df.head()

,ID,Label,Statement,Subject,Speaker,Speaker's job title,state info,party affiliation,barely true count,false count,...,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,2635.json,0.0,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1123.json,0.0,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12465.json,1.0,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5947.json,0.0,When Mitt Romney was governor of Massachusetts...,"history,state-budget",mitt-romney,Former governor,Massachusetts,republican,34.0,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,620.json,1.0,McCain opposed a requirement that the governme...,federal-budget,barack-obama,President,Illinois,democrat,70.0,71.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Get statements and labels

In [ ]:
raw_statement = df['Statement'].tolist()
label = df['Label'].tolist()

#remove last 2 values from the list
raw_statement.remove(raw_statement[4575])
raw_statement.remove(raw_statement[4574])
label.remove(label[4575])
label.remove(label[4574])
print(len(raw_statement))
print(len(label))

4574
4574


In [ ]:
#make a single string
single_string = " ".join(raw_statement)
print(single_string)

Says the Annies List political group supports third-trimester abortions on demand. Health care reform legislation is likely to mandate free sex change surgeries. The Chicago Bears have had more starting quarterbacks in the last 10 years than the total number of tenured (UW) faculty fired during the last two decades. When Mitt Romney was governor of Massachusetts, we didnt just slow the rate of growth of our government, we actually cut it. McCain opposed a requirement that the government buy American-made motorcycles. And he said all buy-American provisions were quote 'disgraceful.'  Women and men both are making less when you adjust for inflation than when John Kitzhaber was first elected governor. I dont know who (Jonathan Gruber) is. ISIS supporter tweeted at 10:34 a.m. Shooting began at 10:45 a.m. in Chattanooga, Tenn. Says Paul Ryan is still endorsing Trump. What (the Obama administration is) going to come out with in the next several months is youre not even going to be able to bu

wordcloud

In [ ]:
# !pip install pillow
# import pandas as pd
# import matplotlib.pyplot as plt
# from wordcloud import WordCloud

# wordcloud1 = WordCloud(width = 3000, height = 2000, stopwords="~", collocations=False).generate((single_string))
# plt.imshow(wordcloud1, interpolation='bilinear')
# plt.axis("off")
# #plt.show()

In [ ]:

from bs4 import BeautifulSoup
import re 
import sys
def clean_str(news):
  news = re.sub(r"∖∖", "", news)
  news = re.sub(r"'", "", news)
  news = re.sub(r"∖", "", news)
  news = re.sub(r"\’", "", news)
  news = re.sub('\n', "", news)
  news = news.lower()
  news = re.sub(' u ', "", news)
  #news = re.sub("'open-quote'","", news)
  return news.strip()


statement = []
for idx in range (len(raw_statement)):
  text_politi = raw_statement[idx]
  statement.append (clean_str(str(text_politi.encode(), 'utf-8'))) #unicode transformation format 8 bit for representing every character
print(statement) 

['says the annies list political group supports third-trimester abortions on demand.', 'health care reform legislation is likely to mandate free sex change surgeries.', 'the chicago bears have had more starting quarterbacks in the last 10 years than the total number of tenured (uw) faculty fired during the last two decades.', 'when mitt romney was governor of massachusetts, we didnt just slow the rate of growth of our government, we actually cut it.', 'mccain opposed a requirement that the government buy american-made motorcycles. and he said all buy-american provisions were quote disgraceful.', 'women and men both are making less when you adjust for inflation than when john kitzhaber was first elected governor.', 'i dont know who (jonathan gruber) is.', 'isis supporter tweeted at 10:34 a.m. shooting began at 10:45 a.m. in chattanooga, tenn.', 'says paul ryan is still endorsing trump.', 'what (the obama administration is) going to come out with in the next several months is youre not e

word_tokenize

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
#tokeize each sentence and store it in a list called tokenized_data
tokenized_data = []
for i in range(len(statement)):
  tokenized_statement =  tokenizer.tokenize(statement[i])
  tokenized_data.append(tokenized_statement)
print(len(tokenized_data))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
4574


stop word removal

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_remove_data = []
for line in tokenized_data:
  temp_stop_remove_statement = []
  for word in line:
    if(word.lower() not in stop_words):
      temp_stop_remove_statement.append(word)
  stop_remove_data.append(temp_stop_remove_statement)
print(stop_remove_data)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[['says', 'annies', 'list', 'political', 'group', 'supports', 'third', 'trimester', 'abortions', 'demand'], ['health', 'care', 'reform', 'legislation', 'likely', 'mandate', 'free', 'sex', 'change', 'surgeries'], ['chicago', 'bears', 'starting', 'quarterbacks', 'last', '10', 'years', 'total', 'number', 'tenured', 'uw', 'faculty', 'fired', 'last', 'two', 'decades'], ['mitt', 'romney', 'governor', 'massachusetts', 'didnt', 'slow', 'rate', 'growth', 'government', 'actually', 'cut'], ['mccain', 'opposed', 'requirement', 'government', 'buy', 'american', 'made', 'motorcycles', 'said', 'buy', 'american', 'provisions', 'quote', 'disgraceful'], ['women', 'men', 'making', 'less', 'adjust', 'inflation', 'john', 'kitzhaber', 'first', 'elected', 'governor'], ['dont', 'know', 'jonathan', 'gruber'], ['isis', 'supporter', 'tweeted', '10', '34', 'shooting', 'began', '10', '45', 'chattanooga', '

wordnet lemmatization

In [ ]:
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
def get_wordnet_pos(word):
  tag = nltk.pos_tag([word])[0][1][0].upper()
  tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
  return tag_dict.get(tag, wordnet.NOUN)

lemmatized_data = []
lemmatizer = WordNetLemmatizer()
print(lemmatizer.lemmatize("going", get_wordnet_pos("going")))
for line in stop_remove_data:
  temp_lemmatized_statement = []
  for word in line:
    temp_lemmatized_statement.append(lemmatizer.lemmatize(word, get_wordnet_pos(word)))
  lemmatized_data.append(temp_lemmatized_statement)
print(lemmatized_data)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
go
[['say', 'annies', 'list', 'political', 'group', 'support', 'third', 'trimester', 'abortion', 'demand'], ['health', 'care', 'reform', 'legislation', 'likely', 'mandate', 'free', 'sex', 'change', 'surgery'], ['chicago', 'bear', 'start', 'quarterback', 'last', '10', 'year', 'total', 'number', 'tenure', 'uw', 'faculty', 'fire', 'last', 'two', 'decade'], ['mitt', 'romney', 'governor', 'massachusetts', 'didnt', 'slow', 'rate', 'growth', 'government', 'actually', 'cut'], ['mccain', 'oppose', 'requirement', 'government', 'buy', 'american', 'make', 'motorcycle', 'say', 'buy', 'american', 'provision', 'quote', 'disgraceful'], ['woman', 'men', 'make', 'less', 'adjust', 'inflation', 'john', 'kitzhaber', 'first', 'elect', 'governor'], ['dont', 'know', 'jonathan', 'gruber'], ['isi', 'supporter', 'tweet', '10', '34', 'shoot', 'begin', '10

wordcloud

In [ ]:
# single_string=""
# for line in lemmatized_data:
#   for word in line:
#     single_string = single_string + " " + word

# print(single_string)
# import pandas as pd
# import matplotlib.pyplot as plt
# from wordcloud import WordCloud

# wordcloud = WordCloud(width = 3000, height = 3000, max_words=200, collocations=False).generate((single_string))
# plt.imshow(wordcloud, interpolation = 'bilinear')
# plt.axis("off")
# plt.show()

fasttext embedding

In [ ]:
print("loading word embeddings")
embedding_index = {}
import codecs
from tqdm import tqdm
import numpy as np
f = codecs.open('/content/drive/MyDrive/mp_hindawi/cc.en.300.vec', encoding='utf-8')
for line in tqdm(f):
  values = line.rstrip().rsplit(' ')
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embedding_index[word] = coefs
f.close()

print("found {} word vectors".format(len(embedding_index)))

loading word embeddings


2000001it [04:01, 8286.11it/s]

found 2000000 word vectors


get the list of list back to a list of strings

In [ ]:
train_data = []
for line in lemmatized_data:
  single_string=""
  single_string = line[0]
  for word in line[1:]:
    single_string = single_string + " " + word
  train_data.append(single_string)
print(train_data)
  

['say annies list political group support third trimester abortion demand', 'health care reform legislation likely mandate free sex change surgery', 'chicago bear start quarterback last 10 year total number tenure uw faculty fire last two decade', 'mitt romney governor massachusetts didnt slow rate growth government actually cut', 'mccain oppose requirement government buy american make motorcycle say buy american provision quote disgraceful', 'woman men make less adjust inflation john kitzhaber first elect governor', 'dont know jonathan gruber', 'isi supporter tweet 10 34 shoot begin 10 45 chattanooga tenn', 'say paul ryan still endorse trump', 'obama administration go come next several month youre even go able burn coal limitedly exist plant', 'federal government think authority regulate toilet seat', 'austin city basically double size every 25 year since found', 'nuclear test conduct nation time earth 2 054th nuclear test five permanent member u n security council conduct 99 99 perce

wordnet tokennizer

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenize = Tokenizer(num_words = 20000) #20000 max words to keep
tokenize.fit_on_texts(train_data) #update the vocabulary if the text is 'the cat is sitting on the floor' then the = 1, cat =2 likewise
sequence = tokenize.texts_to_sequences(train_data) #convert texts into sequence of integers

# Get the total number of unique tokens
word_indexes = tokenize.word_index 

print(word_indexes)
print('Number of Unique Tokens',len (word_indexes))

{'say': 1, 'state': 2, 'year': 3, 'percent': 4, 'tax': 5, 'obama': 6, 'health': 7, 'president': 8, '000': 9, 'job': 10, 'u': 11, 'vote': 12, 'care': 13, 'million': 14, 'people': 15, 'one': 16, 'go': 17, 'new': 18, 'would': 19, 'american': 20, 'time': 21, 'bill': 22, 'government': 23, 'budget': 24, 'country': 25, 'law': 26, 'get': 27, 'every': 28, 'federal': 29, 'rate': 30, 'billion': 31, 'plan': 32, 'texas': 33, '1': 34, 'pay': 35, 'barack': 36, 'republican': 37, 'since': 38, 'last': 39, 'high': 40, 'take': 41, 'wisconsin': 42, 'support': 43, 'make': 44, 'cut': 45, 'first': 46, 'united': 47, 'school': 48, 'money': 49, 'increase': 50, 'cost': 51, 'use': 52, 'two': 53, 'nation': 54, 'work': 55, 'governor': 56, 'want': 57, 'clinton': 58, 'right': 59, 'city': 60, 'raise': 61, 'county': 62, 'three': 63, 'senate': 64, '10': 65, 'debt': 66, 'woman': 67, 'come': 68, 'business': 69, 'insurance': 70, 'national': 71, 'income': 72, 'florida': 73, 'give': 74, 'day': 75, 'create': 76, 'public': 77, 

Real game

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np


data = pad_sequences(sequence, maxlen = 200) #pad the sequences to the same length
y_label = label

#indices_data = np.arange (data.shape[0]) #get an array of numbers which are equal to total rows in the dataset
#print("total number of rows in the dataset is {}".format(len(indices_data))) #total rows check
#np.random.shuffle (indices_data) #shuffle the data
#data = data[indices_data] #select randomly shuffled data\
print(len(data[0]))
print(y_label)

200
[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 

Embedding matrix

In [ ]:
print('preparing embedding matrix')
words_not_found = []
nb_words = len(word_indexes) + 1
embed_dim = 300
import numpy as np
embedding_matrix = np.zeros((nb_words, embed_dim))
for word, i in word_indexes.items():
  if i >= nb_words:
    continue
  embedding_vector = embedding_index.get(word)
  if ((embedding_vector is not None) and len(embedding_vector > 0)):
    embedding_matrix[i] = embedding_vector
  else:
    words_not_found.append(word)

print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
print("sample words not found: ", np.random.choice(words_not_found, 10))

preparing embedding matrix
number of null word embeddings: 338
sample words not found:  ['mopac' 'cornyn' 'lawcaused' 'hillarycare' 'rggi' 'beckhams' 'theislamic'
 'chafin' 'dimassimo' 'hasblown']


model

In [ ]:
print('building model')

from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Lambda, Bidirectional, concatenate, LeakyReLU, Input, Reshape
from tensorflow.keras.models import Model
import pickle
from tensorflow.keras.layers import Embedding, LSTM, GRU, GlobalAveragePooling1D, GlobalMaxPooling1D, AveragePooling1D
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
seq_input = Input(shape=(200, ), dtype='int32')
embedding_layer = Embedding(nb_words, 300, weights=[embedding_matrix], trainable=True)
embedding_sequence = embedding_layer(seq_input)
bi_lstm = Bidirectional(LSTM(units = 50, return_sequences=True))(embedding_sequence)
bi_gru = Bidirectional(GRU(units = 50, return_sequences=True))(embedding_sequence)


max_pool_lstm = GlobalMaxPooling1D()(bi_lstm)
max_pool_gru = GlobalMaxPooling1D()(bi_gru)
avg_pool_lstm = GlobalAveragePooling1D()(bi_lstm)
avg_pool_gru = GlobalAveragePooling1D()(bi_gru)


maxLSTM_maxGRU_concat = concatenate([max_pool_lstm, max_pool_gru])
maxLSTM_avgGRU_concat = concatenate([max_pool_lstm, avg_pool_gru])
avgLSTM_maxGRU_concat = concatenate([avg_pool_lstm, max_pool_gru])
avgLSTM_avgGRU_concat = concatenate([avg_pool_lstm, avg_pool_gru])


maxLSTM_maxGRU_output = Dense(1, activation='sigmoid')(maxLSTM_maxGRU_concat)
maxLSTM_avgGRU_output = Dense(1, activation='sigmoid')(maxLSTM_avgGRU_concat)
avgLSTM_maxGRU_output = Dense(1, activation='sigmoid')(avgLSTM_maxGRU_concat)
avgLSTM_avgGRU_output = Dense(1, activation='sigmoid')(avgLSTM_avgGRU_concat)

maxLSTM_maxGRU_model = Model(seq_input, maxLSTM_maxGRU_output)
maxLSTM_avgGRU_model = Model(seq_input, maxLSTM_avgGRU_output)
avgLSTM_maxGRU_model = Model(seq_input, avgLSTM_maxGRU_output)
avgLSTM_avgGRU_model = Model(seq_input, avgLSTM_avgGRU_output)

maxLSTM_maxGRU_model.summary()
plot_model(to_file = 'maxLSTM_maxGRU_model.png', model = maxLSTM_maxGRU_model, show_shapes='True', show_layer_activations=True)

maxLSTM_avgGRU_model.summary()
plot_model(to_file = 'maxLSTM_avgGRU_model.png', model = maxLSTM_avgGRU_model, show_shapes='True', show_layer_activations=True)

avgLSTM_maxGRU_model.summary()
plot_model(to_file = 'avgLSTM_maxGRU_model.png', model = avgLSTM_maxGRU_model, show_shapes='True', show_layer_activations=True)

avgLSTM_avgGRU_model.summary()
plot_model(to_file = 'avgLSTM_avgGRU_model.png', model = avgLSTM_avgGRU_model, show_shapes='True', show_layer_activations=True)


In [ ]:

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import metrics
from tensorflow.keras.utils import Sequence as seq
from sklearn.utils import class_weight
class_weight = {0:
1.1304960541149944, 1: 0.8965131873044255}
metric = 'val_accuracy'


In [ ]:
# cp=ModelCheckpoint('/content/drive/MyDrive/mp_hindawi/maxLSTM_avgGRU_model.hdf5', monitor=metric, verbose=1,save_best_only=True)

data = np.asarray(data)
label = np.asarray(label)




from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Lambda, Bidirectional, concatenate, LeakyReLU, Input, Reshape
from tensorflow.keras.models import Model
import pickle
from tensorflow.keras.layers import Embedding, LSTM, GRU, GlobalAveragePooling1D, GlobalMaxPooling1D, AveragePooling1D
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model



Max Pooling after LSTM \\
Max Pooling after GRU

In [ ]:

maxLSTM_maxGRU_model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy', 'Precision'])
cp=ModelCheckpoint('/content/drive/MyDrive/mp_hindawi/maxLSTM_maxGRU_model.hdf5', monitor=metric, verbose=1,save_best_only=True)
history=maxLSTM_maxGRU_model.fit(data, label, batch_size = 64, epochs = 15, validation_split=0.2 ,callbacks=[cp], class_weight=class_weight)


Max Pooling after LSTM \\
Average Pooling after GRU

In [ ]:
maxLSTM_maxGRU_model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy', 'Precision'])
cp=ModelCheckpoint('/content/drive/MyDrive/mp_hindawi/maxLSTM_maxGRU_model.hdf5', monitor=metric, verbose=1,save_best_only=True)
history=maxLSTM_maxGRU_model.fit(data, label, batch_size = 64, epochs = 15, validation_split=0.1 ,callbacks=[cp], class_weight=class_weight)

Average Pooling after LSTM \\
Max Pooling after GRU

In [ ]:
avgLSTM_maxGRU_model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy', 'Precision'])
cp=ModelCheckpoint('/content/drive/MyDrive/mp_hindawi/avgLSTM_maxGRU_model.hdf5', monitor=metric, verbose=1,save_best_only=True)
history=avgLSTM_maxGRU_model.fit(data, label, batch_size = 64, epochs = 15, validation_split=0.1 ,callbacks=[cp], class_weight=class_weight)

Average Pooling after LSTM \\
Average Pooling after GRU

In [ ]:
avgLSTM_avgGRU_model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy', 'Precision'])
cp=ModelCheckpoint('/content/drive/MyDrive/mp_hindawi/avgLSTM_avgGRU_model.hdf5', monitor=metric, verbose=1,save_best_only=True)
history=avgLSTM_avgGRU_model.fit(data, label, batch_size = 64, epochs = 15, validation_split=0.1 ,callbacks=[cp], class_weight=class_weight)

Ensemble model \\
Voting between all 4 models described

In [ ]:
#test sample
print("test sample : ", data[0])
print("the result =", end=" ")
print(len(maxLSTM_avgGRU_model.predict(data)))




test sample :  [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    1 3547  793  403  